# Backtest and Analysis Notebook

In [1]:
import pandas as pd
import sys, os
from datetime import datetime, timedelta
from src import generate_signals, compute_pnl
from src.data_loader import fetch_options

# 1) Point to project root
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# 2) Build a multi‐date dataset
start = datetime(2025, 6, 1)
end   = datetime(2025, 7, 17)
all_dfs = []

current = start
while current <= end:
    dt = current.date()
    try:
        df_day = fetch_options("SPY")
        df_day["date"] = dt           # overwrite the snapshot’s date
        all_dfs.append(df_day)
    except Exception as e:
        print(f"Failed on {dt}: {e}")
    current += timedelta(days=1)

full_df = pd.concat(all_dfs, ignore_index=True)

# 3) Save
data_path = os.path.join(project_root, "data", "options_data.csv")
full_df.to_csv(data_path, index=False)

# 4) Reload & backtest
df = pd.read_csv(data_path, parse_dates=["date","expiry"])
df = generate_signals(df)
pnl_df = compute_pnl(df)

# 5) Plot
import matplotlib.pyplot as plt
plt.plot(pnl_df["date"], pnl_df["cumulative_pnl"])
plt.title("Cumulative PnL")
plt.grid(True)
plt.show()


ModuleNotFoundError: No module named 'src'

In [ ]:
# Fetch a live SPY options DataFrame
df = load_options_data("SPY", num_expiries=2)

# Now you can directly call Optopsy on the pandas DF:
stats = op.long_calls(
    df,
    datetime_col="date",
    underlying_price_col="underlying_price",
    option_symbol_col="option_symbol",
    strike_col="strike",
    expiry_col="expiry",
    bid_col="bid",
    ask_col="ask",
    moneyness=(0.99, 1.01),
    days_to_expiry=(5, 30)
)




ValueError: Expected column: {k} not found in DataFrame

In [ ]:
# Cell 3: Run Backtest Example
r = 0.02  # 2% risk-free
sigma = 0.2  # assumed vol
stats = backtest_example(csv_path, r, sigma)
print('Backtest stats (first 5):')
stats.head()
# Plot equity curve
if 'equity_curve' in stats.columns:
    plt.figure(figsize=(10,4))
    plt.plot(stats['date'], stats['equity_curve'])
    plt.title('Equity Curve - Long ATM Calls')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value')
    plt.show()

In [ ]:
# Cell 4: Define Sample Portfolio
demo_portfolio = [
    {'type':'call','strike':100,'expiry':pd.Timestamp('2024-12-20'),'quantity':1,'purchase_price':2.5},
    {'type':'put','strike':95,'expiry':pd.Timestamp('2024-12-20'),'quantity':-1,'purchase_price':1.8}
]

In [ ]:
# Cell 5: Portfolio Heatmap & PnL Curve
hm_fig, pnl_fig = analysis_example(demo_portfolio, r, sigma)
print('Portfolio PnL Heatmap:')
hm_fig.show()
print('Portfolio PnL vs Spot Price:')
pnl_fig.show()

In [ ]:
# Cell 6: Greeks Profile - Delta vs Spot Price
spot_prices = np.linspace(50, 150, 100)
delta_vals = [black_scholes_metric(S, 100, pd.Timestamp('2024-12-20'), r, sigma, 'call', 'Delta') for S in spot_prices]
plt.figure(figsize=(8,4))
plt.plot(spot_prices, delta_vals, label='Call Delta')
plt.axvline(100, linestyle='--', label='Strike')
plt.title('Call Delta vs Spot Price')
plt.xlabel('Spot Price')
plt.ylabel('Delta')
plt.legend()
plt.show()

---
## Interpretation
- **Backtest stats** with cleaner top-level imports.
- **Equity Curve**, **Heatmap**, **P&L Curve**, and **Delta Profile** as before.